# [Richieste ZES Unica](https://www.agenziaentrate.gov.it/portale/documents/20143/6581861/Provvedimento_ZES_UNICA_12_12_2024.pdf)

In [1]:
import camelot
import pandas as pd
import numpy as np

In [29]:
# pdf_path = 'D:\\Zotero\\storage\\2U7RY6GI\\au-2024-270125.pdf' # 2024
pdf_path = 'D:\\Zotero\\storage\\5Z7Q4BLQ\\au-24012025.pdf' # 2025

tables = camelot.read_pdf(pdf_path, pages="all", strip_text="\n", line_scale=40) # Extract tables using Camelot
df = pd.concat([table.df for table in tables], ignore_index=True) # Combine all extracted tables into a single DataFrame
df = df[2:].reset_index(drop=True) # Drop the first two rows
df.columns = ["Numero AU", "Data protocollo", "ID pratica", "Tipologia di intervento", "Comune", "Provincia"]
df["Numero AU"] = df["Numero AU"].astype(int)  # Converte in intero
df["Data protocollo"] = pd.to_datetime(df["Data protocollo"], format="%d/%m/%Y")  # Converte in data

# Funzione per dividere partendo dall'ultima parte
def split_id_pratica(value):
    parts = value.split("-")
    id_value = parts[-1] if len(parts) >= 1 else None
    in_value = pd.to_datetime(parts[-2], format="%d%m%Y", errors="coerce") if len(parts) >= 2 else None
    cf_value = parts[-3] if len(parts) >= 3 else None
    return cf_value, in_value, id_value

df[["CF", "IN", "ID"]] = df["ID pratica"].apply(split_id_pratica).apply(pd.Series) # Applicazione della funzione e creazione delle nuove colonne

comuni = pd.read_csv('province.csv', sep ='\t',na_filter = False)
df = pd.merge(df, comuni[['SIGLA','REGIONE']], left_on='Provincia', right_on='SIGLA', how ='left').drop(columns=['SIGLA'])
df.to_csv('D:/ZES_2025.csv',sep='|')
